# Real time Pupil Detection System

In [1]:
pip install pytest-shutil

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pytest-shutil

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
cv2.__version__

'4.2.0'

In [ ]:
pip uninstall opencv-python

In [1]:
pip install opencv-python==3.4.9.31

  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.2.0.34
    Uninstalling opencv-python-4.2.0.34:
      Successfully uninstalled opencv-python-4.2.0.34
Note: you may need to restart the kernel to use updated packages.


## Pupil detection using CHT with opencv

In [2]:
import cv2
import numpy as np
import os
import shutil
from skimage.transform import resize
from keras.models import Sequential
from keras.models import load_model

Using TensorFlow backend.
C:\Users\Admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [6]:
def pupil_detect(eyeArr):
    rows, cols, _ = eyeArr.shape

    gray_eye = cv2.cvtColor(eyeArr, cv2.COLOR_BGR2GRAY)
    gray_eye = cv2.GaussianBlur(gray_eye, (9, 9), 0)
    gray_eye = cv2.medianBlur(gray_eye, 3)

    threshold = cv2.threshold(gray_eye, 25, 255, cv2.THRESH_BINARY_INV)[1]
    contours = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[1]
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

    for cnt in contours:
        (x, y, w, h) = cv2.boundingRect(cnt)
        cv2.circle(eyeArr, (x + int(w/2), y + int(h/2)), int((h)/3), (0, 0, 255), 3)
        cv2.line(eyeArr, (x + int(w/2), 0), (x + int(w/2), rows), (0, 255, 0), 2)
        cv2.line(eyeArr, (0, y + int(h/2)), (cols , y + int(h/2)), (0, 255, 0), 2)
        break
    
    cv2.imwrite(out_dir + 'res_' + f, eyeArr)
    return eyeArr

input_dir = './samples/'
out_dir = './result/'

#input_dir = 'C:\Users\Admin\Desktop\COEP\SEM 2\DL\Real time pupil detection system\Pupil-Detection-master\pupil_detect_images\samples'
#out_dir = 'C:\Users\Admin\Desktop\COEP\SEM 2\DL\Real time pupil detection system\Pupil-Detection-master\pupil_detect_images\result'


files = [file for file in os.listdir(input_dir) if file.endswith(".jpg")]

if os.path.exists(input_dir + "result") and os.path.isdir(input_dir + "result"):
    shutil.rmtree(input_dir + "result")
if not os.path.exists(input_dir + "result"):
    os.mkdir(input_dir + "result")
    
count = 1
for f in files:
    print("Processing file: ", f)
    pupil_detect(cv2.imread(input_dir + str(f)))
    count += 1
    print("----------Processing Done------------")

Processing file:  1.jpg
----------Processing Done------------
Processing file:  10.jpg
----------Processing Done------------
Processing file:  11.jpg
----------Processing Done------------
Processing file:  12.jpg
----------Processing Done------------
Processing file:  13.jpg
----------Processing Done------------
Processing file:  14.jpg
----------Processing Done------------
Processing file:  15.jpg
----------Processing Done------------
Processing file:  16.jpg
----------Processing Done------------
Processing file:  17.jpg
----------Processing Done------------
Processing file:  18.jpg
----------Processing Done------------
Processing file:  19.jpg
----------Processing Done------------
Processing file:  2.jpg
----------Processing Done------------
Processing file:  20.jpg
----------Processing Done------------
Processing file:  21.jpg
----------Processing Done------------
Processing file:  22.jpg
----------Processing Done------------
Processing file:  23.jpg
----------Processing Done-------

In [11]:
pip install pypng

  Created wheel for pypng: filename=pypng-0.0.20-py3-none-any.whl size=67167 sha256=43198187372a1da83852aab1492fdf118100d473d88c3d1b3ab09e43ca95aa70
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\54\64\43\dfd10cf95dc1687dc5350e861321ecd9a5d76b7c3d96fa1dc6
Successfully built pypng
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import argparse
import math
from scipy.spatial import distance

img = cv2.imread("2.jpg", 0)
img = cv2.medianBlur(img,5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

cv2.namedWindow("Display", flags= cv2.WINDOW_AUTOSIZE)

circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,100,
                           param1=50,param2=25,minRadius=10,maxRadius=30)
circles = np.uint16(np.around(circles))

#coordinates of pupils
point1x = 0.0
point1y = 0.0
point2x = 0.0
point2y = 0.0

#coordinates of object on forehead
leftpointx = 0
leftpointy = 0
rightpointx = 0
rightpointy = 0

#coordinates of pupils in case hough circles doesn't work
leftpointx2 = 0
leftpointy2 = 0
rightpointx2 = 0
rightpointy2 = 0

objectexist = 0

m = 0
k = 0

dist = 0 #distance of pupils in pixels
dist2 = 0 #distance of object in pixels
mm = 100 #distance of object in mm
n = 1

for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
    if n == 1:
        n = 2
        point1x = float(i[0])
        point1y = float(i[1])
    elif n==2 :
        point2x = float(i[0])
        point2y = float(i[1])

print("point1x=",point1x)
print("point1y=",point1y)
print("point2x=",point2x)
print("point2y=",point2y)

dist = math.hypot(point2x - point1x, point2y - point1y) #ipd in pixels

#if dist < 10000:
print("distance = ",dist)

while(1):
    cv2.imshow("Display",cimg)
    if dist2 == 0 and dist < 10000:
        c = cv2.waitKey(0)
        if c == 32: #spacebar
            dist2 = 500
            distanceInMM = dist * (mm / dist2)  # ipd in mm
            print("IPD in mm = " + str(distanceInMM))

    if cv2.waitKey(15)%0x100==27:break		# waiting for clicking escape key
cv2.destroyWindow("Display")

point1x= 460.0
point1y= 630.0
point2x= 268.0
point2y= 232.0
distance =  441.89138937073665
